## Crawl Ted on the English Transcription

Need to add a reconnection mechanism and a page turning function

In [236]:
import requests
from bs4 import BeautifulSoup
import json
import html
import pandas as pd
from tqdm import tqdm

In [240]:
def get_transcript(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    script_tag = soup.find('script', {'type': 'application/ld+json'})
    data = json.loads(script_tag.string)
    return (data['name'],data['transcript'])
def get_talk(url,type_key):
    result = {"type":[],"title":[],"content":[]}
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    script_tag = soup.find('script', id='__NEXT_DATA__')
    json_str = script_tag.string
    json_data = json.loads(json_str)
    for i in tqdm(json_data['props']['pageProps']['talks']):
        try:
            title,content = get_transcript(i['canonicalUrl'])
        except:
            print(i['canonicalUrl'])
            continue
        if len(content) > 5:
            result["type"].append(type_key)
            result["title"].append(title)
            result["content"].append(content)
    return result

In [224]:
type_keyword = 'ai'
url = f"""https://www.ted.com/talks?sort=newest&topics[0]={type_keyword}&language=english"""
result = get_talk(url,type_keyword)

In [241]:
key_wrods_list = ["ai","creativity",'business','collaboration','communication']
df = pd.DataFrame()
for type_keyword in key_wrods_list:
    url = f"""https://www.ted.com/talks?sort=newest&topics[0]={type_keyword}&language=english"""
    try:
        result = get_talk(url,type_keyword)
    except:
        sleep(2)
        print("error:"+type_keyword)
        continue
    pre_df = pd.DataFrame(result)
    df=pd.concat([df, pre_df], ignore_index=True)
    print(type_keyword+"：finish")

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:16<00:00,  1.41it/s]


ai：finish


 71%|██████████████████████████████████████████████████████████                        | 17/24 [00:31<00:48,  6.89s/it]

https://www.ted.com/talks/freada_kapor_klein_how_to_use_venture_capital_for_good


 83%|████████████████████████████████████████████████████████████████████▎             | 20/24 [00:34<00:11,  2.94s/it]

...

https://www.ted.com/talks/storybook_maze_a_street_librarian_s_quest_to_bring_books_to_everyone


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:10<00:00,  2.20it/s]


collaboration：finish


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:11<00:00,  2.01it/s]

communication：finish


In [243]:
df.to_excel("Transcription_v1.0.xlsx")